In [1]:
from TagBert import TagBert

In [2]:
import pandas as pd
import os

In [3]:
data_dir='../datasets'
fname='meta_list.json'
path=os.path.join(data_dir,fname)

In [4]:
df=pd.read_json(path)
df=df[df['has_lyric']==1]

In [5]:
import torch
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer

In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [25]:
def encode(tokenizer,text):
    if isinstance(text,str):
        if text:
            text=[text]
    ret=[]
    for ele in text:
        ele=tokenizer.cls_token+ele+tokenizer.eos_token
        ele=tokenizer.encode(ele)
        ret.append(ele)
    return ret


In [26]:
def embed(model,encoded):
    if not isinstance(encoded,list):
        encoded=[encoded]
    ret=[]
    for ele in encoded:
        if len(ele) >512: 
            ele=ele[:512]
        token_ids=torch.tensor(ele).unsqueeze(0)
        output=model(token_ids)[0]
        output=output[:,0,:]
        ret.append(output)
    return ret

In [27]:
tagbert=TagBert(model,tokenizer,encode,embed)

In [28]:
from utils import DataUtils
client_id='A4Mceclx8PV864nLzpJA'
client_secret='gSGhYL3Roj'
utils=DataUtils(client_id,client_secret)

In [29]:
for title,lyric in zip(df['track_title'].iloc[:20],df['lyric'].iloc[:20]):
    if utils.translator.get_language_type(lyric) == 'en': continue
    tags=tagbert.extract_tag(lyric,keyphrase_ngram_range=(1, 2), stop_words=None,top_n=10)
    print(title)
    print(tags)

밤편지
[[('말이에요 우리의', 0.6579), ('반딧불을 당신의', 0.6117), ('좋을까요', 0.6109), ('띄울게요 좋은', 0.5887), ('좋을까요 파도가', 0.5808), ('글씨처럼 그대가', 0.5724), ('사랑한다는 그날의', 0.5709), ('여기 마음속에', 0.5697), ('있다면', 0.561), ('입맞춤을 떠올려', 0.5533)]]
모든 날, 모든 순간 (Every day, Every Moment)
[[('눈물이나 힘든', 0.599), ('고단했던 나의', 0.596), ('미래지만 품속에', 0.5824), ('품속에', 0.5568), ('불안했던 나의', 0.5382), ('너의 품은', 0.5362), ('내게 왔던', 0.5267), ('따뜻했어 고단했던', 0.5256), ('품속에 있는', 0.5203), ('나의', 0.52)]]
첫눈처럼 너에게 가겠다
[[('삶은 따뜻했었다고', 0.6729), ('살아가라 내가', 0.6531), ('가겠다 너에게', 0.6362), ('너는 울었다', 0.6242), ('울게 만들었을까', 0.621), ('나를 부를', 0.6196), ('짧은 마주침이', 0.603), ('작은 숨결로', 0.5989), ('빗물처럼 너는', 0.587), ('찬란한 것을', 0.5788)]]
벚꽃 엔딩
[[('자장노래 어떤가요', 0.6406), ('걸어요 거리를', 0.6278), ('퍼질 거리를', 0.5961), ('떨림과 둘이', 0.5957), ('걸어요 봄바람', 0.5907), ('불면 울렁이는', 0.5853), ('거리를 둘이', 0.5843), ('휘날리며', 0.5832), ('좋아요 봄바람', 0.5809), ('둘이 걸어요', 0.5797)]]
봄날
[[('영원할 없으니까', 0.7131), ('계절도 영원할', 0.7119), ('뿐이야 8월에도', 0.7084), ('보게 될까', 0.6995), ('만나게 될까', 0.6974), ('끝내

In [30]:
for title,lyric in zip(df['track_title'].iloc[:20],df['lyric'].iloc[:20]):
    if utils.translator.get_language_type(lyric) == 'en': continue
    tags=tagbert.extract_tag(lyric,keyphrase_ngram_range=(1, 1), stop_words=None,top_n=10)
    print(title)
    print(tags)

밤편지
[[('좋을까요', 0.6109), ('있다면', 0.561), ('당신의', 0.5508), ('띄울게요', 0.5449), ('마음속에', 0.5372), ('나의', 0.5259), ('행운이', 0.4827), ('사라져', 0.4693), ('입맞춤을', 0.4688), ('떠올려', 0.4666)]]
모든 날, 모든 순간 (Every day, Every Moment)
[[('품속에', 0.5568), ('나의', 0.52), ('나를', 0.5107), ('날에', 0.4817), ('따뜻했어', 0.4758), ('왔던', 0.4724), ('유일한', 0.4577), ('있는', 0.4476), ('너에게로', 0.445), ('휴식처', 0.4448)]]
첫눈처럼 너에게 가겠다
[[('나를', 0.5716), ('숨결로', 0.5674), ('따뜻했었다고', 0.5579), ('너는', 0.5449), ('부를', 0.5334), ('울었다', 0.5267), ('고운', 0.5241), ('닿은', 0.5223), ('순간들을', 0.5222), ('질투도', 0.5046)]]
벚꽃 엔딩
[[('휘날리며', 0.5832), ('흩날리는', 0.5438), ('저편에서', 0.5265), ('어떤가요', 0.5224), ('벚꽃', 0.5199), ('잎이', 0.5175), ('자장노래', 0.5174), ('들려오는', 0.5102), ('밤에', 0.506), ('봄바람', 0.5055)]]
봄날
[[('영원할', 0.6514), ('멀어져요', 0.6497), ('솔직히', 0.6357), ('떠도는', 0.6275), ('원망하기보단', 0.6252), ('얼마나', 0.6242), ('8월에도', 0.6208), ('될까', 0.6174), ('어떤', 0.6112), ('못하는데', 0.6108)]]
너를 만나
[[('떠올리며', 0.6221), ('나의', 0.5995), ('행복해져', 0.5917), ('영원하고', 0